<a href="https://colab.research.google.com/github/fgabim19/Curso-Ciencia-de-Datos/blob/main/GuiaTensorflow_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

tf.keras.backend.clear_session()

Introduccion
Ya estás familiarizado con el uso del metodo keras.Sequential() para crear modelos. La API funcional es una forma de crear modelos mas dinamicos que con Sequential: La API funcional puede manejar modelos con topología no lineal, modelos con capas compartidas y modelos con múltiples entradas o salidas.

Se basa en la idea de que un modelo de aprendizaje profundo suele ser un gráfico acíclico dirigido (DAG) de capas. La API funcional es un conjunto de herramientas para construir gráficos de capas.

Considera el siguiente modelo:

(input: 784-vectores dimensionales)<br>
        ↧<br>
[Dense (64 units, activacion relu)]<br>
       ↧<br>
[Dense (64 units, activacion relu)]<br>
       ↧<br>
[Dense (10 units, activacion softmax)]<br>
       ↧<br>
(output: distribución de probabilidad en 10 clases)

### Entrenamiento, evaluación e inferencia.

In [ ]:
from tensorflow.keras import layers

In [ ]:
inputs = keras.Input(shape=(784,), name='img')
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

In [ ]:
model.summary()

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = keras.optimizers.RMSprop(),
              metrics = ['accuracy'])
history = model.fit(
    x_train, y_train,
    batch_size=64,
    epochs=5,
    validation_split=0.2)

test_score = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

### Almacenado y serialización

In [ ]:
model.save('path_to_my_model.h5')
del model

model = keras.models.load_model("path_to_my_model.h5")

### Manipulación de topologías gráficas complejas

La API funcional facilita la manipulación de múltiples entradas y salidas. Esto no se puede manejar con la API secuencial.

Aquí hay un ejemplo simple.

Supongamos que está creando un sistema para clasificar los tickets de emisión personalizados por prioridad y enrutarlos al departamento correcto.

Tu modelo tendrá 3 entradas:

* Título del ticket (entrada de texto)
* Cuerpo del texto del ticket (entrada de texto)
* Cualquier etiqueta agregada por el usuario (entrada categórica)

Tendrá dos salidas:

* Puntuación de prioridad entre 0 y 1 (salida sigmoidea escalar)
* El departamento que debe manejar el ticket (salida softmax sobre el conjunto de departamentos)

Construyamos este modelo en pocas líneas con la API funcional.

In [ ]:
num_tags = 12
num_words = 10000
num_departments = 4

title_input = keras.Input(shape=(None,), name='title')
body_input = keras.Input(shape=(None,), name='body')
tags_input = keras.Input(shape=(num_tags,), name='tags')

title_features = layers.Embedding(num_words, 64)(title_input)
body_features = layers.Embedding(num_words, 64)(body_input)

title_features = layers.LSTM(128)(title_features)
body_features = layers.LSTM(32)(body_features)

x = layers.concatenate([title_features, body_features, tags_input])

priority_pred = layers.Dense(1, activation="sigmoid", name='priority')(x)

departament_pred = layers.Dense(num_departments, activation='softmax', name='department')(x)

model = keras.Model(inputs = [title_input, body_input, tags_input],
                    outputs=[priority_pred, departament_pred])


In [ ]:
keras.utils.plot_model(model, "multi_input_and_ouput_model.png", show_shapes=True)

Al compilar este modelo, podemos asignar diferentes pérdidas a cada salida. Incluso puede asignar diferentes pesos a cada pérdida, para modular su contribución a la pérdida total de entrenamiento.

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(1e-3),
              loss=['binary_crossentropy', 'categorical_crossentropy'],
              loss_weights=[1., 0.2])

In [ ]:
#Como dimos nombres a nuestras capas de salida, también podríamos especificar la pérdida de esta manera:
model.compile(optimizer=keras.optimizers.RMSprop(1e-3),
              loss={'priority': 'binary_crossentropy',
                    'department': 'categorical_crossentropy'},
              loss_weights=[1., 0.2])

### Extendiendo la API escribiendo capas personalizadas

In [ ]:
class CustomDense(layers.Layer):

    def __init__(self, units=32):
        super(CustomDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
inputs = keras.Input((4,))
outputs = CustomDense(10)(inputs)


model = keras.Model(inputs, outputs)

#### Valida su modelo mientras lo está definiendo.